# Connect to Odrive

In [ ]:
import odrive
from odrive.enums import *
import time
import math
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

my_drive = odrive.find_any()
if my_drive is not None:
    print('Connected!')
    print('my_drive serial {}'.format(my_drive.serial_number))
else:
    print('Not connected')
    

In [ ]:
m0 = my_drive.axis0
m1 = my_drive.axis1

# Configure output limits

In [ ]:
curr_limit = 2 #A
my_drive.axis0.motor.config.current_lim = curr_limit
my_drive.axis1.motor.config.current_lim = curr_limit

current_limit = (curr_limit, curr_limit)

my_drive.axis0.controller.config.vel_limit = 1000000
my_drive.axis1.controller.config.vel_limit = 1000000

# Calibration

In [ ]:
m0.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
m1.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE

In [ ]:
m0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
m1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

In [ ]:
m0.requested_state = AXIS_STATE_IDLE
m1.requested_state = AXIS_STATE_IDLE

# Define transmission matrix

In [ ]:
import sympy as sp
from sympy.utilities.lambdify import lambdify

(thetaL_sym, thetaR_sym) = sp.symbols("""thetaL_sym thetaR_sym """, real = True)
(x_cartesian, y_cartesian) = sp.symbols("""x_cartesian y_cartesian """, real = True)

def T(theta, x, y):
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), x], 
                      [sp.sin(theta), sp.cos(theta), y],
                      [0, 0, 1]])
def car_polar(x, y):
    r = sp.sqrt(x**2+y**2)
    theta = sp.atan(y/x)
    return sp.Matrix([[r],
                      [theta]])
def sym_to_np(car_polar):
    return np.array(car2polar).astype(np.float64)

def sym_to_np(T):
    return np.array(T).astype(np.float64)


# Polar FK Through transformation matrices

In [ ]:
l1 = 0.09
l2 = 0.16
w = 0.07

x_r = -w/2 + l1*sp.cos(thetaR_sym)
x_l = w/2 + l1*sp.cos(thetaL_sym)

y_r = l1*sp.sin(thetaR_sym)
y_l = l1*sp.sin(thetaL_sym)

theta3_sym = sp.atan2(y_l - y_r, x_l - x_r)
L = sp.sqrt((x_l - x_r)**2 + (y_l - y_r)**2)

FK = T(thetaL_sym, w/2, 0)@T((sp.pi - thetaL_sym + theta3_sym), l1, 0)@sp.Matrix([L/2, -sp.sqrt(l2**2 - (L/2)**2), 1])
FK = FK[:2,:]
FK.simplify()
FK_fast = lambdify((thetaR_sym, thetaL_sym), FK) # FK_fast(x,y)

J = FK.jacobian([thetaR_sym, thetaL_sym]).evalf()
J_fast = lambdify((thetaR_sym, thetaL_sym), J) # J_fast(x,y)

#convert coordinate to polar
FK_polar = car_polar(FK[0], FK[1])
FK_polar_fast = lambdify((thetaR_sym, thetaL_sym), FK_polar) # FK_polar_fast(x,y)

# #arctan is defined between -pi/2~pi/2 
# if FK_polar_fast(theta_R, theta_L)[1] < 0
#     FK_polar_fast(theta_R, theta_L)[1] = FK_polar_fast(theta_R, theta_L)[1] + np.pi
# else
#     FK_polar_fast(theta_R, theta_L)[1] = FK_polar_fast(theta_R, theta_L)[1]

J_polar = FK_polar.jacobian([thetaR_sym, thetaL_sym]).evalf()
J_polar_fast = lambdify((thetaR_sym, thetaL_sym), J_polar) # J_polar_fast(x,y)

# Define home location (which is zero angle)

In [ ]:
my_drive.axis0.requested_state = AXIS_STATE_IDLE
my_drive.axis1.requested_state = AXIS_STATE_IDLE

In [ ]:
m0_home_pos = m0.encoder.pos_estimate
m1_home_pos = m1.encoder.pos_estimate

theta_R_home = -((m0_home_pos) / 8192 * 2 * np.pi) + np.pi
theta_L_home = -((m1_home_pos) / 8192 * 2 * np.pi)

theta_R_home_d = theta_R_home / (2 * np.pi) * 360
theta_L_home_d = theta_L_home / (2 * np.pi) * 360

print(theta_R_home_d)
print(theta_L_home_d)

# Define equilibrium toe location (where you want to start)

In [ ]:
m0_equilibrium_toe_pos = m0.encoder.pos_estimate
m1_equilibrium_toe_pos = m1.encoder.pos_estimate

theta_R_eq = -((m0_equilibrium_toe_pos - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
theta_L_eq = -((m1_equilibrium_toe_pos - m1_home_pos) / 8192 * 2 * np.pi)

theta_R_eq_d = theta_R_eq / (2 * np.pi) * 360
theta_L_eq_d = theta_L_eq / (2 * np.pi) * 360

print(theta_R_eq_d)
print(theta_L_eq_d)

# Define polar compliance

In [ ]:
# stiffness coeff
kr = 10
ktheta = 10
k = np.array([[kx,0],[0,ky]])
# damping coeff
cr = 2
ctheta = 2
c = np.array([[cx,0],[0,cy]])
# motor torque const
Kt = 0.0285

def polar_compliance(theta_R, theta_L, vel_R, vel_L):
    
    J_trans = -np.transpose(J_polar_fast(theta_R, theta_L))
    disp = FK_polar_fast(theta_R, theta_L) - FK_polar_fast(theta_R_eq, theta_L_eq) 
    vel = J_polar_fast(theta_R, theta_L) @ np.array([[vel_R], [vel_L]])
    
    currents = J_trans @ ( k @ disp + c @ vel )*(1/Kt)

    return currents 

# Control loop

In [ ]:
# m0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
# m1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
my_drive.axis0.requested_state = AXIS_STATE_IDLE
my_drive.axis1.requested_state = AXIS_STATE_IDLE
    

t = 10
theta_R = np.zeros(10*t)
theta_L = np.zeros(10*t)

for i in range(10*t):
    #read the encoder position & velocity
    m0_encoder_pos = m0.encoder.pos_estimate
    m1_encoder_pos = m1.encoder.pos_estimate
    m0_encoder_vel = m0.encoder.vel_estimate
    m1_encoder_vel = m0.encoder.vel_estimate
    #current angle(rad) & velocity(rad/s)
    theta_R = - ((m0_encoder_pos - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
    theta_L = - ((m1_encoder_pos - m1_home_pos) / 8192 * 2 * np.pi)
    vel_R = - ((m0_encoder_vel) / 8192 * 2 * np.pi)
    vel_L = - ((m1_encoder_vel) / 8192 * 2 * np.pi)
    
    #current angle(degree)
    theta_R_d = theta_R / (2 * np.pi) * 360
    theta_L_d = theta_L / (2 * np.pi) * 360
        
    print(FK_polar_fast(theta_R, theta_L)[1])


    #get cureent
    current = polar_compliance(theta_R, theta_L, vel_R, vel_L)
    #make sure the current is not exceeds the limit
    current_0 = min(np.abs(current[0]), current_limit[0]) * np.sign(current[0])
    current_1 = min(np.abs(current[1]), current_limit[1]) * np.sign(current[1]) 
    
    print(current_0,current_1)
      
    #control the motors
    my_drive.axis0.controller.set_current_setpoint(-current_0)
    my_drive.axis1.controller.set_current_setpoint(-current_1)
    
    time.sleep(0.1)
    
my_drive.axis0.requested_state = AXIS_STATE_IDLE
my_drive.axis1.requested_state = AXIS_STATE_IDLE
    